In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible.revnet import ResidualBlock, invert, SubsampleSplitter, ViewAs, ReversibleBlockOld
from spectral_norm import spectral_norm
from conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:

plt.plot(np.sin(np.linspace(0, 8*np.pi, endpoint=False, num=8) + np.pi/2))

In [ ]:
np.sin(np.linspace(0, 6*np.pi, endpoint=False, num=8))

In [ ]:
rng = RandomState(39482014)


plt.plot(np.sin(np.linspace(0, 4*np.pi, endpoint=False, num=8)))
#plt.plot(np.sin(np.linspace(0, 4*np.pi, endpoint=False, num=16)))
amps_a = np.abs(rng.randn(320) + 2)
phases_a = rng.rand(320) * 2 * np.pi
signals = np.array([amp * np.sin(np.linspace(0, 8*np.pi, endpoint=False, num=8) + np.pi/2+ phase) 
     for amp, phase in zip(amps_a, phases_a)])
plt.figure()
plt.plot(signals[:4].squeeze().T)
inputs = [np_to_var(signals[:,None,:,None], dtype=np.float32).cuda()]

In [ ]:
from matplotlib.lines import Line2D
plt.figure(figsize=(10,6))
for i_class in range(len(inputs)):
    plt.plot(var_to_np(inputs[i_class].squeeze()).T, color=seaborn.color_palette()[i_class],lw=0.5);
lines = [Line2D([0], [0], color=seaborn.color_palette()[i_class],) for i_class in range(2)]
plt.legend(lines, ['Right', 'Rest',], bbox_to_anchor=(1,1,0,0))
plt.title('Input signals')

In [ ]:
def rev_block(n_c, n_i_c):
     return ReversibleBlockOld(
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True)),
         
        nn.Sequential(
        nn.Conv2d(n_c // 2, n_i_c,(3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c // 2,(3,1), stride=1, padding=(1,0),bias=True))
    )
    
def res_block(n_c, n_i_c):
     return ResidualBlock(
        nn.Sequential(
        nn.Conv2d(n_c, n_i_c, (3,1), stride=1, padding=(1,0),bias=True),
        nn.ReLU(),
            nn.Conv2d(n_i_c, n_c, (3,1), stride=1, padding=(1,0),bias=True)),
    )

In [ ]:
from discriminator import ProjectionDiscriminator
from reversible.revnet import SubsampleSplitter, ViewAs
from reversible.util import set_random_seeds
from reversible.revnet import init_model_params
from torch.nn import ConstantPad2d
import torch as th
from conv_spectral_norm import conv_spectral_norm
from disttransform import DSFTransform



set_random_seeds(2019011641, True)
feature_model = nn.Sequential(
    SubsampleSplitter(stride=[2,1],chunk_chans_first=False),
    rev_block(2,32),
    rev_block(2,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True),
    rev_block(4,32),
    rev_block(4,32),
    SubsampleSplitter(stride=[2,1],chunk_chans_first=True),
    rev_block(8,32),
    rev_block(8,32),
    ViewAs((-1,8,1,1), (-1,8))
)
feature_model.cuda()


In [ ]:
from ampphase import outs_to_amp_phase

In [ ]:
_ = [p.data.zero_() for p in feature_model.parameters()]

In [ ]:
outs = feature_model(inputs[0])

In [ ]:
outs[0].squeeze()

In [ ]:
inputs[0][0].squeeze()

In [ ]:
x1,x2 = th.chunk(outs, 2, dim=1,)

In [ ]:
x1 = th.mean(x1, dim=1, keepdim=True)
x2 = th.mean(x2, dim=1, keepdim=True)
amps, phases  = outs_to_amp_phase(th.cat((x1,x2), dim=1))

In [ ]:
inputs[0][0].squeeze()

In [ ]:
inputs[0][3].squeeze()

In [ ]:
np.fft.rfft(var_to_np(inputs[0][0].squeeze()))

In [ ]:
np.abs(np.fft.rfft(var_to_np(inputs[0][0].squeeze())))

In [ ]:
th.sqrt(th.sum(inputs[0][0].squeeze() ** 2))

In [ ]:
th.sum(th.abs(inputs[0][0].squeeze()))

In [ ]:
plt.plot(var_to_np(amps))
plt.plot(amps_a)